In [76]:
from tensorflow import keras
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 


import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)


In [77]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("GPU Enabled")
    except RuntimeError as e:
        print(e)

In [79]:
import re
curr_dir = 'data'
all_goals = []
for filename in os.listdir(curr_dir):
    try:
        objFile = open(os.path.join(curr_dir,filename), "r")
        fileContent = objFile.read()
        start = fileContent.find("GOAL") + len("GOAL")
        end = fileContent.find("== DATA") or fileContent.find("==== DATA")
        substring = fileContent[start:end]
        all_goals.append((substring.replace('\n','').strip('==')))
    except:
        pass
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in all_goals]
corpus[:10]

['the main objective of to predict bankruptcy based on financial data bases utilizing prefinancial knowledge and experience of financial experts',
 'to discover new laws of physics',
 'a new system that is able to automatically decipher a lost language without needing advanced knowledge of its relation to other languages',
 'perform an unlicensed experimental test of selfdriving vehicles ',
 'autonomous trucks',
 'developing a biocompatible implantable ai platform that classifies in real time healthy and pathological patterns in biological signals such as heartbeats ',
 'research was done to reveal sleep quality using machine learning models without the need for questionnaires surveys emails etc',
 'to detect all instances of a category in an image and for each instance mark the pixels that belong to it this is called simultaneous detection and segmentation sds datainput data is a random image with several instances that can be distinguished and resolved into separate individual instan

In [80]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[2, 196],
 [2, 196, 566],
 [2, 196, 566, 3],
 [2, 196, 566, 3, 1],
 [2, 196, 566, 3, 1, 23],
 [2, 196, 566, 3, 1, 23, 1458],
 [2, 196, 566, 3, 1, 23, 1458, 46],
 [2, 196, 566, 3, 1, 23, 1458, 46, 14],
 [2, 196, 566, 3, 1, 23, 1458, 46, 14, 333],
 [2, 196, 566, 3, 1, 23, 1458, 46, 14, 333, 15]]

In [81]:
max_sequence_len = max([len(x) for x in inp_sequences])
input_sequences = np.array(pad_sequences(inp_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label =  tf.keras.utils.to_categorical(label, num_classes=total_words)
print(max_sequence_len)
print(input_sequences)

869
[[   0    0    0 ...    0    2  196]
 [   0    0    0 ...    2  196  566]
 [   0    0    0 ...  196  566    3]
 ...
 [   0    0    0 ... 2225  565 1074]
 [   0    0    0 ...  565 1074   36]
 [   0    0    0 ... 1074   36  813]]


In [82]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 868, 10)           51090     
                                                                 
 lstm_2 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 5109)              516009    
                                                                 
Total params: 611,499
Trainable params: 611,499
Non-trainable params: 0
_________________________________________________________________


In [83]:
model.fit(predictors, label, epochs=100, verbose=5)


Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [103]:
# Saving the model:
#model.save('unsupervised_model')

champion_model = tf.keras.models.load_model('unsupervised_model')

In [104]:
def text_generator(starting_text,number_of_words,model,max_sequence_len):
    for _ in range(number_of_words):
        token_list = tokenizer.texts_to_sequences([starting_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predict_x=model.predict(token_list, verbose=0) 
        predicted=np.argmax(predict_x,axis=1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        starting_text += ' ' + output_word
    return starting_text



'machine learning selects secret vibrations from data from earthquakestechnology can allow scientists to map large underground geological structures for example oil reserves geothermal sources etc more accurately with these vibrations on the survival state to face to face interaction teachers can be used for the robot to fit a model is an model to the most likely of the investigation they results to create a model to analyse if an model also interesting in the most results results it is able to understand the use machine learning algorithms which is able to the utilisation of the data add the experiment add a model model as a test and test the model model as the model in the amount of rainfall through the sound produced this paper used to the input port of the accuracy of the experiment acoustic produce used to the features and the experiment set add the model add the model add the model add the model module this is very dataset he can be able to test the model score model that evaluat

In [106]:
prompt = 'Enter the start of the text generation?\n'
i=input(prompt)

Enter the start of the text generation?
machine learning


In [109]:
text_generator(i,200,champion_model,max_sequence_len)

'machine learning selects secret vibrations from data from earthquakestechnology can allow scientists to map large underground geological structures for example oil reserves geothermal sources etc more accurately with these vibrations on the survival state to face to face interaction teachers can be used for the robot to fit a model is an model to the most likely of the investigation they results to create a model to analyse if an model also interesting in the most results results it is able to understand the use machine learning algorithms which is able to the utilisation of the data add the experiment add a model model as a test and test the model model as the model in the amount of rainfall through the sound produced this paper used to the input port of the accuracy of the experiment acoustic produce used to the features and the experiment set add the model add the model add the model add the model module this is very dataset he can be able to test the model score model that evaluat

machine learning
